
# Bayesian Optimization with Retrieval Optimizer

This notebook demonstrates how to use Bayesian optimization to tune Redis-based retrieval pipelines. Unlike a grid study—which tests all combinations—Bayesian optimization intelligently searches the configuration space, prioritizing promising settings based on previous results. This is especially useful when the number of possible configurations is large and exhaustive search would be too costly.

You'll define a study configuration, specify embedding models and search methods, and let the optimizer guide the search toward the best-performing retrieval setup.


## Dataset

We'll import a dataset from the [beir benchmark IR project](https://github.com/beir-cellar/beir) to get going quickly. 



In [ ]:
# Load data
from redis_retrieval_optimizer.corpus_processors import eval_beir

# check the link above for different datasets to try
beir_dataset_name = "nfcorpus"

# Load sample data
corpus, queries, qrels = eval_beir.get_beir_dataset(beir_dataset_name)

## Study config

In this directory there is a yaml file containing a configuration for a bayesian study that looks like this:

```yaml
# path to data files for easy read
corpus: "data/nfcorpus_corpus.json"
queries: "data/nfcorpus_queries.json"
qrels: "data/nfcorpus_qrels.json"

index_settings:
  name: "optimize"
  vector_field_name: "vector" # name of the vector field to search on
  text_field_name: "text" # name of the text field for lexical search
  from_existing: false
  vector_dim: 384 # should match first embedding model or from_existing
  additional_fields:
      - name: "title"
        type: "text"

optimization_settings:
  # defines the options optimization can take
  metric_weights:
    f1_at_k: 1
    embedding_latency: 1
    total_indexing_time: 1
  algorithms: ["hnsw"]
  vector_data_types: ["float16", "float32"]
  distance_metrics: ["cosine"]
  n_trials: 10
  n_jobs: 1
  ret_k: [1, 10] # potential range of value to be sampled during study
  ef_runtime: [10, 20, 30, 50]
  ef_construction: [100, 150, 200, 250, 300]
  m: [8, 16, 64]


search_methods: ["vector", "lin_combo"]
embedding_models:
  - type: "hf"
    model: "sentence-transformers/all-MiniLM-L6-v2"
    dim: 384
    embedding_cache_name: "vec-cache" # avoid names with including 'ret-opt' as this can cause collisions
    dtype: "float32"
```

## Running a study

To run a study simple pass the path to config, redis_url, and corpus processing function to the `run_bayes_study` function and the package will take care of the rest. 

In [1]:
# run study

# add root redis_retrieval_optimizer to path until available on pypi
import sys
import os

# Get the current notebook directory
current_dir = os.path.dirname(os.path.abspath(''))

# Go up two directory levels (adjust the number as needed)
parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# Add the parent directory to the Python path if it's not already there
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    print(f"Added {parent_dir} to Python path")


import os
from redis_retrieval_optimizer.bayes_study import run_bayes_study
from redis_retrieval_optimizer.corpus_processors import eval_beir
from dotenv import load_dotenv

# load environment variables containing necessary credentials
load_dotenv()

redis_url = os.environ.get("REDIS_URL", "redis://localhost:6379/0")

metrics = run_bayes_study(
    config_path="bayes_study_config.yaml",
    redis_url=redis_url,
    corpus_processor=eval_beir.process_corpus
)

/Users/robert.shelton/Library/Caches/pypoetry/virtualenvs/redis-retrieval-optimizer-OYoDaORV-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-05-14 14:01:37,473] A new study created in memory with name: test


14:01:37 redisvl.index.index INFO   Index already exists, overwriting.
14:01:39 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
14:01:39 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.27it/s]

14:01:40 root INFO   Recreating index...


14:01:41 root INFO   Corpus size: 3633
14:01:45 root INFO   Data indexed total_indexing_time=3.559s, num_docs=3633


...

Batches: 100%|██████████| 1/1 [00:00<00:00, 53.51it/s]

14:02:44 root INFO   Recreating index...


14:02:44 root INFO   Corpus size: 3633
14:02:50 root INFO   Data indexed total_indexing_time=4.804s, num_docs=3633
14:02:52 root INFO   Saving metrics for study: 789c70ba-3208-44b1-9c06-db1888162d55, METRICS={'search_method': ['hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'vector', 'vector', 'hybrid', 'vector', 'hybrid'], 'total_indexing_time': [3.559, 3.361, 3.126, 3.094, 3.443, 2.742, 4.808, 2.471, 1.896, 4.804], 'avg_query_time': [0.004628060402884941, 0.004614599718028916, 0.00321132922689243, 0.004435610106854985, 0.0029207316714543676, 0.001332445410383006, 0.0019653323265052063, 0.0032455301137162434, 0.001478021727995976, 0.004497922980010325], 'model': ['sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-Mini

[I 2025-05-14 14:02:52,248] Trial 9 finished with value: 4.804 and parameters: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'hybrid', 'algorithm': 'hnsw', 'var_dtype': 'float16', 'distance_metric': 'cosine', 'ret_k': 5, 'ef_runtime': 50, 'ef_construction': 200, 'm': 64}. Best is trial 6 with value: 4.808.


14:02:52 root INFO   Completed Bayesian optimization... 


14:02:52 root INFO   Best Configuration: 6: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'vector', 'algorithm': 'hnsw', 'var_dtype': 'float16', 'distance_metric': 'cosine', 'ret_k': 10, 'ef_runtime': 30, 'ef_construction': 200, 'm': 64}:


14:02:52 root INFO   Best Score: [4.808]




In [2]:
metrics[["search_method", "algorithm", "vector_data_type", "ef_construction", "ef_runtime", "m", "avg_query_time", "recall@k", "precision", "ndcg@k"]].sort_values(by="ndcg@k", ascending=False)

,search_method,algorithm,vector_data_type,ef_construction,ef_runtime,m,avg_query_time,recall@k,precision,ndcg@k
1,hybrid,hnsw,float16,200,30,8,0.004615,0.154988,0.243344,0.203648
0,hybrid,hnsw,float16,200,50,8,0.004628,0.154988,0.243344,0.203624
2,hybrid,hnsw,float32,250,50,16,0.003211,0.154988,0.243344,0.203624
3,hybrid,hnsw,float16,100,10,16,0.004436,0.154988,0.243344,0.203624
4,hybrid,hnsw,float32,300,30,16,0.002921,0.154988,0.243344,0.203624
7,hybrid,hnsw,float32,150,50,16,0.003246,0.154988,0.243344,0.203624
9,hybrid,hnsw,float16,200,50,64,0.004498,0.154988,0.243344,0.203624
6,vector,hnsw,float16,200,30,64,0.001965,0.152715,0.243034,0.193432
5,vector,hnsw,float32,200,20,16,0.001332,0.148293,0.240867,0.189538
8,vector,hnsw,float32,100,10,16,0.001478,0.127060,0.227864,0.166708
